## import

In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack
import cv2

import tensorflow as tf
import tflearn
from tflearn.layers.normalization import batch_normalization
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d

%matplotlib inline

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data Manager Class

In [2]:
class DataManager():
    
    def __init__(self):
        self.N = 256
        self.dif = 128
        self.window = np.hamming(self.N)
        self.im_width = 128
        
    def voice2images(self, data, folder_name = "images"):
        i = 0
        _wav = self.N + self.dif * (self.im_width - 1)
        _dif = int(_wav/2)
        while(True):
            _data = data[i*_dif:i*_dif + _wav]
            if (len(_data) < _wav):
                break
            self.save_image(self.spec2image(_data), folder_name + "/" + str(i) + ".png")
            i = i + 1
    
    def images2voice(self, image_num = 10000, folder_name = "images"):
        _wav = self.N + self.dif * (self.im_width - 1)
        _dif = int(_wav/2)
        voice = np.zeros(_dif * (image_num+1))
        for i in range(image_num):
            data = self.read_image(folder_name + "/" + str(i) + ".png")
            if data is None:
                break
            voice[i*_dif:i*_dif+_wav] = voice[i*_dif:i*_dif+_wav] + self.image2spec(data)*np.r_[np.linspace(0,1,_dif), np.linspace(1,0,_dif)]
        
        """
        voice = []
        for i in range(image_num):
            data = self.read_image(folder_name + "/" + str(i) + ".png")
            voice.append(self.image2spec(data))
        """
        return voice
        
    def spec2image(self, data):
        image = []
        for i in range(self.im_width):
            image.append(self.dct(data[self.dif*i:self.dif*i + self.N]))
        return image
    
    def image2spec(self, data):
        voice = np.zeros(self.N + self.dif*(self.im_width-1))        
        vsum = np.zeros(self.N + self.dif*(self.im_width-1))
        for i in range(self.im_width):
            voice[self.dif*i:self.dif*i + self.N] = voice[self.dif*i:self.dif*i + self.N] + self.idct(data[i]) * self.window
            vsum[self.dif*i:self.dif*i + self.N] +=self.window ** 2 
        pos = (vsum != 0)
        voice[pos] /= vsum[pos]
        return voice
        
    def dct(self, data):       
        data = data * self.window
        data = scipy.fftpack.dct(data)
        pm = np.sign(data)
        data = np.log(np.abs(data)+0.0001)
        data = pm * data
        data = data[:int(self.N/2)]
        data = data / 20
        return data
    
    def idct(self, data):
        data = data * 20
        pm = np.sign(data)
        data = np.exp(np.abs(data))
        data = pm * data
        data = np.append(data, np.zeros(int(self.N/2)))
        data = scipy.fftpack.idct(data) / (2 * len(data))
        return data

    def read_wave(self, filename):
        wf = wave.open(filename , "r" )
        buf = wf.readframes(wf.getnframes())
        data = np.frombuffer(buf, dtype="int16") # 16 kHz?
        return data    
    
    def save_wave(self, data, filename):
        wf = wave.open(filename, "w")
        wf.setnchannels(2)
        wf.setsampwidth(2) #2 byte = 16 bit
        wf.setframerate(16000)
        wf.writeframes(np.array(data,  dtype="int16"))
        wf.close()
        
    def save_image(self, data, file_name):
        data = np.array(data) * 255
        cv2.imwrite(file_name, data)
        
    def read_image(self, file_name):
        data = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
        return data / 255

## Data init

In [3]:
dm = DataManager()

In [ ]:
voice1 = dm.read_wave("VoiceData/kamiya.wav")
dm.voice2images(voice1, "ImageData/Kamiya_images")

voice2 = dm.read_wave("VoiceData/kizuna.wav")
dm.voice2images(voice2, "ImageData/Kizuna_images")

In [ ]:
image = dm.spec2image(voice1)
plt.imshow(image)

In [ ]:
re_voice = dm.image2spec(image)
voice = dm.images2voice(2000, "ImageData/Kamiya_images")
dm.save_wave(voice, "test.wav")

## Network

In [ ]:
class DiscoGAN():
    
    def __init__(self, sess):
        
        self.sess = sess
                    
        self.x_1 = tflearn.input_data(shape=[None, 128, 128, 1])
        self.x_2 = tflearn.input_data(shape=[None, 128, 128, 1])
        
        self.gen_learning_rate = 0.01
        self.dis_learning_rate = 0.01
        
        self.param_gan = 0.1
        self.param_feat = 0.9
        self.param_recon = 1.
        
        self._create_optimizer()
        
        sess.run(tf.global_variables_initializer())
    
    def create_gen_network(self, network_name, inputs, reuse=None):
        with tf.variable_scope(network_name, reuse=reuse):
            
            network = conv_2d(inputs, 64, 1,  activation='relu')
            #network = batch_normalization(network)
            network = conv_2d(network, 128, 1, activation='relu')
            #network = batch_normalization(network)
            network = conv_2d(network, 256, 1, activation='relu')
            #network = batch_normalization(network)
            network = conv_2d(network, 512, 1, activation='relu')
            
            network = conv_2d(network, 512, 1,  activation='relu')
            #network = batch_normalization(network)
            network = conv_2d(network, 128, 1,  activation='relu')
            #network = batch_normalization(network)
            network = conv_2d(network, 64, 1,  activation='relu')
            #network = batch_normalization(network)
            outputs = conv_2d(network, 1, 1,  activation='relu')
            
        return outputs
        
    def create_dis_network(self, network_name, inputs, reuse=None):
        with tf.variable_scope(network_name, reuse=reuse):
            
            network = conv_2d(inputs, 64, 1,  activation='relu')
            network = conv_2d(network, 128, 1, activation='relu')
            network = conv_2d(network, 256, 1, activation='relu')
            network = conv_2d(network, 512, 1, activation='relu')
            #network = fully_connected(network, 256, activation='relu')
            outputs = fully_connected(network, 1, activation='sigmoid')
            
        return outputs, network
    
    def compute_loss_gan(self, y_real, y_fake):
        """
        batchsize = y_real.shape[0]
        loss_dis = F.sum(F.softplus(-y_real) + F.softplus(y_fake))
        loss_gen = F.sum(F.softplus(-y_fake))
        return loss_dis / batchsize, loss_gen / batchsize
        """ 
        loss_dis =  tflearn.mean_square(y_real, tf.ones(tf.shape(y_real))) + tflearn.mean_square(y_fake, tf.zeros(tf.shape(y_real)))
        loss_gen = tflearn.mean_square(y_fake, tf.ones(tf.shape(y_real)))
        return loss_dis, loss_gen

    def compute_loss_feat(self, feats_real, feats_fake):
        """
        losses = 0
        for feat_real, feat_fake in zip(feats_real, feats_fake):
        feat_real_mean = tf.reduce_sum(feat_real, 0) / feat_real.shape[0]
        feat_fake_mean = tf.reduce_sum(feat_fake, 0) / feat_fake.shape[0]
        l2 = (feat_real_mean - feat_fake_mean) ** 2
        loss = tf.reduce_sum(l2) / l2.size
        losses += loss
        return losses
        """
        return tflearn.mean_square(feats_real, feats_fake)
        
    
    def _create_optimizer(self):

        self.x_12 = self.create_gen_network("G1", self.x_1)
        self.x_21 = self.create_gen_network("G2", self.x_2)

        self.x_121 = self.create_gen_network("G2", self.x_12, reuse=True)
        self.x_212 = self.create_gen_network("G1", self.x_21, reuse=True)

        recon_loss_1 = tflearn.mean_square(self.x_1, self.x_121)
        recon_loss_2 = tflearn.mean_square(self.x_2, self.x_212)

        y_1_real, feats_1_real = self.create_dis_network("D1", self.x_1)
        y_1_fake, feats_1_fake = self.create_dis_network("D1", self.x_21, reuse=True)

        y_2_real, feats_2_real = self.create_dis_network("D2", self.x_2)
        y_2_fake, feats_2_fake = self.create_dis_network("D2", self.x_12, reuse=True)

        feat_loss_1 = self.compute_loss_feat(feats_1_real, feats_1_fake)
        feat_loss_2 = self.compute_loss_feat(feats_2_real, feats_2_fake)
        
        gan_loss_dis_1, gan_loss_gen_1 = self.compute_loss_gan(y_1_real, y_2_fake)
        gan_loss_dis_2, gan_loss_gen_2 = self.compute_loss_gan(y_2_real, y_2_fake)
        
        # gan = 0.1
        # feat = 0.9
        # recon = 1

        total_loss_gen_1 = self.param_gan*gan_loss_gen_2 + self.param_feat*feat_loss_2 + self.param_recon*recon_loss_1
        total_loss_gen_2 = self.param_gan*gan_loss_gen_1 + self.param_feat*feat_loss_1 + self.param_recon*recon_loss_2

        self.gen_loss = total_loss_gen_1 + total_loss_gen_2
        self.dis_loss = gan_loss_dis_1 + gan_loss_dis_2
        
        self.gen_loss = recon_loss_1
        self.dis_loss = recon_loss_2
        
        self.gen_optimize = tf.train.AdamOptimizer(self.gen_learning_rate).minimize(
            loss=self.gen_loss
        )
        self.dis_optimize = tf.train.AdamOptimizer(self.dis_learning_rate).minimize(
            loss=self.dis_loss
        )
        
    def optimize(self, x_1, x_2):
        self.sess.run(self.gen_optimize, feed_dict={
            self.x_1: x_1,
            self.x_2: x_2
        }) 
        self.sess.run(self.dis_optimize, feed_dict={
            self.x_1: x_1,
            self.x_2: x_2
        }) 
        
    def gen1(self, image):
        return self.sess.run(self.x_12, feed_dict={
            self.x_1: image,
        }) 

    def gen2(self, image):
        return self.sess.run(self.x_21, feed_dict={
            self.x_2: image,
        }) 

## Train & Test

In [ ]:
def train(sess, discoGAN, epoch):
        
        for i in range(epoch):
            if (i+1 % 10 == 0):
                print(str(i) + "/" + str(epoch))
            if (i+1 % 100 == 0):
                test(sess, discoGAN, "ImageData/Kamiya_images/100.png")
            image1 = dm.read_image("ImageData/Kamiya_images/" + str(i) + ".png")
            image2 = dm.read_image("ImageData/Kizuna_images/" + str(i) + ".png")
            image1 = np.reshape(image1, (-1, 128, 128, 1))
            image2 = np.reshape(image2, (-1, 128, 128, 1))
            
            discoGAN.optimize(image1, image2)
            
def test(sess, discoGAN, file_name):
        image = dm.read_image(file_name)
        gen_image = discoGAN.gen1(np.reshape(image, (-1, 128, 128, 1)))
        gen_image = np.reshape(gen_image, (128, 128))
        
        plt.title("original")
        plt.imshow(image)
        plt.show()
        plt.title("generated")
        plt.imshow(gen_image)
        plt.show()

In [ ]:
with tf.Session() as sess:
        
    discoGAN = DiscoGAN(sess)
    train(sess, discoGAN, 5000)
    test(sess, discoGAN, "ImageData/Kamiya_images/100.png")
    
    saver = tf.train.Saver()
    saver.save(sess, "ModelData/model.ckpt")